In [ ]:
# ============================================================
# 支持向量机 (Support Vector Machine, SVM)
# ============================================================
# 
# SVM 是一种强大的分类算法，核心思想是：
# 找到一个超平面，使得两类样本之间的间隔(margin)最大化
# 
# 关键概念：
# 1. 超平面 (Hyperplane): 分隔不同类别的决策边界
# 2. 支持向量 (Support Vectors): 距离超平面最近的样本点
# 3. 间隔 (Margin): 超平面到最近样本点的距离
# 
# 硬间隔 vs 软间隔：
# - 硬间隔：要求完全分开，不允许任何错误
# - 软间隔：允许一些违规，更鲁棒
# 
# 正则化参数 C：
# - C 大：严格分类，可能过拟合
# - C 小：更大的间隔，允许更多违规
# ============================================================

from sklearn import datasets
import numpy as np

# 设置随机种子，确保结果可复现
np.random.seed(42)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# ============================================================
# 第一步：加载和准备数据
# ============================================================

# 加载鸢尾花数据集
iris = datasets.load_iris()

# 只使用两个特征：花瓣长度和花瓣宽度
X = iris['data'][:, (2, 3)]  # (petal length, petal width)

# 二分类：是否为 Virginica 类
y = (iris['target'] == 2).astype(np.float64)

print(f"数据形状: X = {X.shape}")
print(f"类别分布: Virginica = {sum(y)}, 非Virginica = {len(y) - sum(y)}")

# ============================================================
# 第二步：创建 SVM Pipeline
# ============================================================
# 
# 重要：SVM 对特征尺度敏感，必须先标准化！

svm_clf = Pipeline([
    # 标准化：将特征缩放到均值=0，标准差=1
    ('scaler', StandardScaler()),
    
    # 线性 SVM 分类器
    # C=1: 正则化参数，控制间隔和违规之间的权衡
    # loss='hinge': 使用标准的 hinge 损失函数
    ('linear_svc', LinearSVC(C=1, loss='hinge', random_state=42))
])

# 训练模型
svm_clf.fit(X, y)

print("\nPipeline 结构:")
for name, step in svm_clf.steps:
    print(f"  {name}: {type(step).__name__}")

In [ ]:
# ============================================================
# 第三步：使用模型进行预测
# ============================================================

# 测试样本：花瓣长度=5.5cm, 花瓣宽度=1.7cm
test_samples = [[5.5, 1.7], [2.0, 0.5], [4.0, 1.3]]

print("预测结果:")
print("-" * 50)
print(f"{'花瓣长度':<10} {'花瓣宽度':<10} {'预测类别':<15}")
print("-" * 50)

for sample in test_samples:
    pred = svm_clf.predict([sample])[0]
    class_name = "Virginica" if pred == 1 else "非Virginica"
    print(f"{sample[0]:<10} {sample[1]:<10} {class_name:<15}")

print("-" * 50)

In [ ]:
# ============================================================
# 第四步：可视化决策边界
# ============================================================

import matplotlib.pyplot as plt

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 创建网格
x0_min, x0_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
x1_min, x1_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx0, xx1 = np.meshgrid(np.linspace(x0_min, x0_max, 200),
                        np.linspace(x1_min, x1_max, 200))

# 预测网格点
X_grid = np.c_[xx0.ravel(), xx1.ravel()]
y_pred = svm_clf.predict(X_grid).reshape(xx0.shape)

# 获取决策函数值（用于绘制间隔边界）
decision_values = svm_clf.decision_function(X_grid).reshape(xx0.shape)

plt.figure(figsize=(10, 6))

# 绘制决策边界和间隔
plt.contourf(xx0, xx1, y_pred, alpha=0.3, cmap='RdYlGn')
plt.contour(xx0, xx1, decision_values, levels=[-1, 0, 1], 
            colors=['blue', 'black', 'blue'], linestyles=['--', '-', '--'])

# 绘制训练样本
plt.scatter(X[y==0, 0], X[y==0, 1], c='red', marker='o', 
            edgecolors='black', s=50, label='非Virginica')
plt.scatter(X[y==1, 0], X[y==1, 1], c='green', marker='s', 
            edgecolors='black', s=50, label='Virginica')

plt.xlabel('花瓣长度 (cm)', fontsize=12)
plt.ylabel('花瓣宽度 (cm)', fontsize=12)
plt.title('SVM 线性分类器 - 决策边界与间隔', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# ============================================================
# 知识总结
# ============================================================
print("\n" + "=" * 60)
print("支持向量机 (SVM) 总结")
print("=" * 60)
print("""
1. 核心思想：最大间隔分类器
   - 找到使间隔最大化的超平面
   - 只有支持向量决定边界位置

2. 关键参数：
   - C: 正则化参数（C大=硬间隔，C小=软间隔）
   - kernel: 核函数类型 ('linear', 'rbf', 'poly')
   - gamma: RBF核的参数

3. 使用建议：
   - 必须标准化特征
   - 对高维数据效果好
   - 样本量大时训练较慢

4. 常用类：
   - LinearSVC: 线性SVM，训练快
   - SVC: 支持核技巧，更灵活
   - SVR: SVM回归
""")